# Problem Set 1

##### Set up environment

In [40]:
# Set working directory
dir = "/Users/JoshuaHigbee/Box/2. Second Year/2. Winter Quarter - 2021/" *
      "Industrial Organization II - Hortacsu/Problem Sets/Problem Set 1/";
cd(dir);

# Load packages
using CSV, DataFrames, Random, Distributions, LinearAlgebra,
      LatexPrint, StatsBase, Plots, SpecialFunctions
using Optim, ForwardDiff, PyCall

# Set seed
Random.seed!(12345);

# Read in data
data = CSV.read("psetOne.csv", DataFrame);
println(names(data))

["Market", "Constant", "Price", "EngineSize", "SportsBike", "Brand2", "Brand3", "z1", "z2", "z3", "z4", "shares"]


## Question 8

##### Load data for market $t=17$, and set up parameter values (coefficient order is Price, Constant, Engine CC, BikeType, Brand2, Brand3) and $\xi$ values.

In [41]:
# Data 
d17 = data[data.Market .== 17, :];

# Parameters
θ_given =  [-3.0 1.0 1.0 2.0 -1.0 1.0]'
α = θ_given[1]

# Ownership matrix
d17.Brand1 = Ref(1.0) .- max.(d17.Brand2, d17.Brand3);
Δ = (d17.Brand1 .* d17.Brand1') .+ (d17.Brand2 .* d17.Brand2') .+ (d17.Brand3 .* d17.Brand3')

# ξ values
ξ_rand = rand(Normal(0,1),7);
ξ_newton = ξ_rand; # ξ_simple
ξ_newton = zeros(7);

##### Define useful functions for solving fixed point pricing equation.

In [42]:
# Shares function
function s_newton(price)
    X = hcat(price, d17.Constant, d17.EngineSize, d17.SportsBike, d17.Brand2, d17.Brand3)
    exp_term = X * θ_given .+ ξ_newton
    max_exp = maximum(exp_term)
    num = exp.(exp_term .- max_exp)
    return num / (exp(-max_exp) + sum(num))
end


# Jacobian of shares function
function s_Jac_newton(price)
    s = s_newton(price)
    return - α * ((s * s') .- diagm(vec(s))) # α is negative in this setting
end


# Hessian of shares function
function s_Hess_newton(price)
    s = s_newton(price)
    Hess = Array{Float64}(undef, 7, 7, 7)
    for j in 1:7
        for k in 1:7
            for l in 1:7
                Hess[j,k,l] = α^2 *
                      ((2*s[j]*s[k]*s[l] - 3*s[j]*s[k] + s[j])*(j==k && j==l) +
                      (2*s[j]*s[k]*s[l] - s[j]*s[l])*(j==k && j!=l) +
                      (2*s[j]*s[k]*s[l] - s[k]*s[l])*(j!=k && j==l) +
                      (2*s[j]*s[k]*s[l] - s[j]*s[k])*(j!=k && k==l) +
                      (2*s[j]*s[k]*s[l])*(l!=k && k!=j && j!=l))
            end
        end
    end
    return Hess
end


# Function for Newton's method (using p + Ω^{-1} * s = 0)
function f_newton(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    Ω = Δ .* s_J
    f = price + inv(Ω) * s
    return f
end


# Jacobian of Ω^{-1} (for use in computing Jacobian of function for fxp est)
function Ω_inv_Jac_newton(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    s_H = s_Hess_newton(price)
    Ω = Δ .* s_J
    Ω_J = Δ .* s_H

    Ω_J_inv = Array{Float64}(undef, length(s), length(s), length(s))
    for j in 1:length(s)
        Ω_J_inv[:,:,j] = - inv(Ω) * Ω_J[:,:,j] * inv(Ω)
    end
    return Ω_J_inv
end


# Jacobian of function for Newton's method
function f_newton_J(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    Ω = Δ .* s_J
    Ω_inv_J = Ω_inv_Jac_newton(price)
    Ω_inv_J_times_s = Matrix(0.0 * I, length(s), length(s))
    for j in 1:length(s)
        Ω_inv_J_times_s[:,j] = Ω_inv_J[:,:,j] * s
    end
    ∇f_newton = I + Ω_inv_J_times_s + inv(Ω) * s_J
    return ∇f_newton
end


# Actually estimate the fixed point
function newton_fxp(price, ϵ)
    diff = 1
    p_0 = price
    iter=1
    while diff > ϵ
        # Solve equation
        fxp = f_newton(p_0)
        ∇fxp = f_newton_J(p_0)
        # ∇fxp = ForwardDiff.jacobian(f_newton, p_0)
        p_1 = p_0 - inv(∇fxp) * fxp

        # Update values and loop
        diff = maximum(abs.(p_1 .- p_0))
        println("Iteration " * string(iter) * "     Difference " * string(diff))
        p_0 = p_1
        iter = iter + 1
    end
    return p_0
end

newton_fxp (generic function with 1 method)

##### Solve fixed point equation.

In [43]:
p_init = ones(7);
p_init = d17.Price;
p_optimal = newton_fxp(p_init, 1.0e-10)

println(" ")
p_optimal

Iteration 1     Difference 2.484826536163365
Iteration 2     Difference 0.2893243176990721
Iteration 3     Difference 0.0726965625358531
Iteration 4     Difference 0.006524393561025077
Iteration 5     Difference 4.2079754713597595e-5
Iteration 6     Difference 1.679278494037817e-9
Iteration 7     Difference 6.661338147750939e-16
 


7×1 Array{Float64,2}:
 0.38447413297903904
 0.3844741329790391
 0.3523194580438232
 0.3523194580438232
 1.7822602606218438
 1.782260260621844
 1.782260260621844

##### Check to ensure it is a fixed point

In [44]:
f_newton(p_optimal)
show(stdout, "text/plain", f_newton(p_optimal))

7×1 Array{Float64,2}:
 -1.1102230246251565e-16
  0.0
  0.0
 -5.551115123125783e-17
  3.9968028886505635e-15
  3.9968028886505635e-15
  3.3306690738754696e-15

## Question 9

##### Write share prediction functions.

In [45]:
# Individual share predictor function (given mean utilities)
function sHat_ind(;δ, X=0.0, σ, ζ=0.0, I_n=1)
    
    # Fill values if unspecified (default)
    if X == 0.0
        X = repeat(fill(0.0,length(σ))', length(δ))
    end
    if ζ == 0.0
        ζ = reshape(fill(0.0,length(σ)*I_n), I_n, length(σ))
    end

    # Construct numerators
    const_term = repeat(δ', outer=I_n)
    rc_term = ζ * (X .* repeat(σ, outer=length(δ)))'
    exp_term = const_term .+ rc_term
    max_exp = maximum(exp_term, dims=2)
    num = exp.(exp_term - repeat(max_exp, inner=(1,length(δ))))

    # Construct denominators and shares
    denom = repeat(exp.(-max_exp) .+ sum(num, dims=2), inner=(1,length(δ)))
    ŝ_i = num ./ denom
    return ŝ_i
end

# Share predictor function (for heterogeneity)
function sHat(;δ, X=0.0, σ, ζ=0.0, I_n=1)
    ŝ_i = sHat_ind(δ=δ, X=X, σ=σ, ζ=ζ, I_n=I_n)
    ŝ = 1/(I_n) * sum(ŝ_i, dims=1)
    return vec(ŝ)
end

# Share predictor function for taking ŝ after it's already computed
function sHat_from_ŝ(; ŝ_i, I_n=1)
    ŝ = 1/(I_n) * sum(ŝ_i, dims=1)
    return vec(ŝ)
end

sHat_from_ŝ (generic function with 1 method)

##### Set up parameters and data for test cases.

In [46]:
J_q9 = [3, 3, 3];
δ_q9 = [zeros(J_q9[1]), [40, 20, 20], zeros(J_q9[3])];
σ_q9 = [[0.0], [0.0], [0.1]];
X_q9 = [[], [], [repeat([1.0], 3), [1.0; 3.0; -1.0], [10.0; 10.0; -3.0]]];
I_q9 = [[], [], 50];
ζ_q9 = [[], [], reshape(rand(Normal(0,1),length(σ_q9[3])*I_q9[3]), I_q9[3], length(σ_q9[3]))];
shares_q9 = [];

##### Test all cases

In [47]:
# First case
s_test = sHat(δ=δ_q9[1], σ=σ_q9[1]);
println(s_test);
sum(s_test, dims=2)[1]
push!(shares_q9, s_test);

[0.25, 0.25, 0.25]


In [48]:
# Second case
s_test = sHat(δ=δ_q9[2], σ=σ_q9[2]);
println(s_test);
sum(s_test, dims=2)[1]
push!(shares_q9, s_test);

[0.9999999958776926, 2.061153613941849e-9, 2.061153613941849e-9]


In [49]:
# Third case
s_test = sHat(δ=δ_q9[3], X=X_q9[3][1], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

s_test = sHat(δ=δ_q9[3], X=X_q9[3][2], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

s_test = sHat(δ=δ_q9[3], X=X_q9[3][3], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

[0.2499492, 0.2499492, 0.2499492]
[0.247299, 0.2547225, 0.2504074]
[0.2492555, 0.2492555, 0.2680796]


## Question 10

##### Create functions for share inversion.

In [113]:
# Contraction mapping function
function s_contraction_map(;s, X, I_n, σ, ζ, J, L̄=1.0-1.0e-7, output=false)
      δ_0 = convert.(Real, zeros(length(s)))
      L = 0.5 # Initialize contraction mapping ratio
      iter = 0 # Initialize iteration count
      δ_diff = 0.5 # Initialize difference in δ (in case L misbehaves)
      δ_diff_h = Array{Real}(undef, 1, 3)
      while (δ_diff > 1.0e-7) && (L < L̄)
            # Numerical inversion
            δ_1 = δ_0 .+ log.(s) .- log.(sHat(δ=δ_0, X=X, σ=σ, ζ=ζ, I_n=I_n))

            # Max distance in δ
            δ_diff = maximum(abs.(δ_1 - δ_0))

            # Track iterations and update Lipschitz constant every 50 iters
            iter = iter + 1
            if ((iter + 2) % 50) == 0
                  δ_diff_h[1] = Real(δ_diff)
            end
            if ((iter + 1) % 50) == 0
                  δ_diff_h[2] = Real(δ_diff)
            end
            if (iter % 50) == 0
                  δ_diff_h[3] = Real(δ_diff)
                  L = (δ_diff_h[2]^2) / (δ_diff_h[3] * δ_diff_h[1])
                  if output==true
                        println("Iteration  " * string(iter) *
                              "    Contraction term L is " * string(L))
                  end
            end
            δ_0 = δ_1
      end

      return δ_0
end


# Log shares Jacobian function - vectorized
function log_s_Jac(; ŝ_i, I_n=1, J)
      ∫ŝ_j_mat = repeat(1/(I_n) .* sum(ŝ_i, dims=1), outer=J)'
      own_term = 1.0/(I_n) * ŝ_i' * (Ref(1.0) .- ŝ_i) .* Matrix(I, J, J)
      cross_term = -1.0/(I_n) * (ŝ_i' * ŝ_i) .* (Ref(1.0) .- Matrix(I, J, J))
      return (own_term .+ cross_term) ./ ∫ŝ_j_mat
end


# Newton's method function
function s_newton_fxp(; s, δ_0, X, I_n, σ, ζ, J, ϵ_conv=1.0e-14, output=false, maxiter = 1e5)
    diff = 0.5
    iter = 0 # Initialize iteration count
    while diff > ϵ_conv && iter < maxiter
        # Compute
        ŝ_i = sHat_ind(δ=δ_0, X=X, σ=σ, ζ=ζ, I_n=I_n)
        ŝ = sHat_from_ŝ(ŝ_i=ŝ_i, I_n=I_n)
        log_sJ = log_s_Jac(ŝ_i=ŝ_i, I_n=I_n, J=J)
        log_s = log.(ŝ)
        δ_1 = δ_0 .- inv(log_sJ) * (log_s - log.(s))

        # Check difference and iterate
        diff = maximum(abs.(δ_1 .- δ_0))
        iter = iter + 1
        if output==true
              println("Iteration  " * string(iter) *
                    "    Difference is " * string(diff))
        end
        δ_0 = δ_1
    end

    return δ_0
end


# Combine two methods function
function invert_s(; s, X, I_n, σ, ζ, J, L̄=1.0-1.0e-7, ϵ_conv=1.0e-14, output=false, maxiter_newton = 1e5)
    δ_contraction = s_contraction_map(s=s, X=X, I_n=I_n, σ=σ, ζ=ζ, J=J, L̄=L̄, output=output)
    δ = s_newton_fxp(s=s, δ_0=δ_contraction, X=X, I_n=I_n, σ=σ, ζ=ζ, J=J, ϵ_conv=ϵ_conv, output=output, 
            maxiter = maxiter_newton)
    return δ
end

invert_s (generic function with 1 method)

##### Test functions with data from market 17.

In [51]:
I_17 = 25;
X_17 = convert(Matrix, hcat(d17[:, [:Price, :Constant, :EngineSize, :SportsBike, :Brand2, :Brand3]]));
σ_17 = zeros(size(X_17)[2])';
ζ_17 = reshape(rand(Normal(0,1),length(σ_17)*I_17), I_17, length(σ_17));

In [52]:
# Test contraction
δ_contract = s_contraction_map(s=d17.shares, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1])

7-element Array{Float64,1}:
  2.0701830111983366
 -0.34354547915214617
 -2.9269583105221644
  0.6072436591254329
  3.1415591256443003
  1.3887023174228195
  2.3476939453562755

In [53]:
# Test Newton's method
s_newton_fxp(s=d17.shares, δ_0 = δ_contract, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1], ϵ_conv=1.0e-15)

7-element Array{Float64,1}:
  2.070264979033952
 -0.3434635113165314
 -2.9268763426865503
  0.6073256269610476
  3.1416410934799153
  1.3887842852584344
  2.347775913191892

In [66]:
# Test both
invert_s(s=d17.shares, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1], L̄=1.0-1.0e-7, ϵ_conv=1.0e-15)

7-element Array{Float64,1}:
  2.070264979033952
 -0.3434635113165314
 -2.9268763426865503
  0.6073256269610476
  3.1416410934799153
  1.3887842852584344
  2.347775913191892

##### Test with data from the previous question.

In [55]:
println(invert_s(s=shares_q9[1], σ=σ_q9[2], X=0.0, I_n=1, ζ=0.0, J=J_q9[1]))
println(invert_s(s=shares_q9[2], σ=σ_q9[2], X=0.0, I_n=1, ζ=0.0, J=J_q9[2]))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][1], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][2], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][3], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))

[0.0, 0.0, 0.0]
[36.91942667635634, 16.919426676356338, 16.919426676356338]
[0.0, 0.0, 0.0]
[0.000167855770944695, -0.02965908149334844, -0.012089398619322995]
[-0.06117617872123282, -0.06117617872123282, -0.14113460280480175]


## Question 11

##### Write function for computing $\xi$

In [106]:
# Implicit function for ξ - all markets (X_all is vector of matrices)
function ξ_rclogit(; X_all, θ, s_all, ζ=0.0, I_num=1, maxiter_newton=1e5, rclogit_output=false)
    # θ split between linear terms (1) and nonlinear terms (2)
    αβ = θ[1] # Put price first for consistency
    σ = θ[2]
    
    # Loop through markets and invert shares
    δhat_all = []
    for mkt in 1:length(X_all)
        if rclogit_output==true
            println("Market number is " * string(mkt) * " of " * string(length(X_all)))
        end
        δhat = invert_s(s=s_all[mkt], X=X_all[mkt], I_n=I_num, σ=σ, ζ=ζ, J=length(s_all[mkt]), 
                    L̄=1.0-1.0e-7, ϵ_conv=1.0e-14, maxiter_newton = maxiter_newton)
        push!(δhat_all, δhat)
    end
    
    # Stack data and compute δ_mean
    δhat = reduce(vcat, δhat_all)
    X_stack = reduce(vcat, X_all)
    δ_mean = X_stack * αβ

    # Subtract off mean utility to get ξ̂
    return δhat .- δ_mean
end

ξ_rclogit (generic function with 1 method)

##### Write function for splicing data into markets (array of matrices and vectors, for shares and variables).

In [57]:
function mkt_X_s_break(; exclude_vec)
    X_full_mkt = []
    s_full_mkt = []
    for mkt in unique(data[[x ∉ exclude_vec for x in data.Market], :Market])
        X_mkt = convert(Matrix, data[data.Market .== mkt,
                    [:Price, :Constant, :EngineSize, :SportsBike, :Brand2, :Brand3]])
        push!(X_full_mkt, X_mkt)
        
        s_mkt = data[data.Market .== mkt, :shares]
        push!(s_full_mkt, s_mkt)
    end
    return [X_full_mkt, s_full_mkt]
end

mkt_X_s_break (generic function with 1 method)

##### Test functions with data from market.

In [93]:
# Create test parameters
I_test = 25
σ_test = zeros(6)
ζ_test = reshape(rand(Normal(0,1),length(σ_test)*I_test), I_test, length(σ_test))
Z_test = convert(Matrix, data[:, [:z1, :z2, :z3, :z4]])
W_test = inv(Z_test' * Z_test);

# Choose data
test_data = mkt_X_s_break(exclude_vec = []);

In [72]:
# Test functions at θ = 0
θ_zeros = [zeros(6), zeros(6)']
@time ξ_zeros = ξ_rclogit(X_all=test_data[1], θ=θ_test, s_all=test_data[2], ζ=ζ_test, I_num=I_test);

 48.793324 seconds (363.40 M allocations: 40.777 GiB, 19.28% gc time)


In [112]:
ξ_zeros[1:8]

8-element Array{Float64,1}:
  3.4175577069186143
  2.2199898323356324
 -2.8348206236877442
  4.317272874619498
  1.459620984134158
  1.0254328494996467
  1.536865084873692
 -2.7318744786927476

##### Write GMM objective function using implicit function $\xi$.

In [63]:
function f_gmm_rclogit_implθ(; W=I, Z, X_all, θ, s_all, ζ, I_num)
    ξ = ξ_rclogit(X_all=X_all, θ=θ, s_all=s_all, ζ=ζ, I_num=I_num)
    return (Z' * ξ)' * W * (Z' * ξ)
end

# Test objective function - computing ξ within
f_gmm_rclogit_implθ(W=W_test, Z=Z_test, X_all=test_data[1], θ=[zeros(6), zeros(6)'], s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test)

252.5932803130474

##### Rewrite objective function as a function of only $\sigma$ (no linear terms)

In [74]:
# Function for computing linear parameters
function θ_tsls(; X, Z, W, δ)
    B = X' * Z * W * Z'
    return inv(B * X) * (B * δ)
end


# Function returning ξ(σ), δ(σ), and θ̄(σ)
function ξδθ_σ_rclogit(; X_all, σ, s_all, ζ=0.0, I_num=1, Z, W)
    δ = []
    for mkt in 1:length(X_all)
        δ_mkt = invert_s(s=s_all[mkt], X=X_all[mkt], I_n=I_num, σ=σ, ζ=ζ, J=length(s_all[mkt]))
        push!(δ, δ_mkt)
    end
    δ_σ_stack = reduce(vcat, δ)
    X_stack = reduce(vcat, X_all)

    αβ = θ_tsls(X=X_stack, Z=Z, W=W, δ=δ_σ_stack)

    ξ = δ_σ_stack .- (X_stack * αβ)
    return (ξ = ξ, δ_all = δ, αβ = αβ)
end


# Rewrite objective function as only a function of σ
function f_gmm_rclogit_σ(;W, Z, X_all, σ, s_all, ζ, I_num)
    output = ξδθ_σ_rclogit(X_all=X_all, σ=σ, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W)
    ξ = output.ξ
    return (Z' * ξ)' * W * (Z' * ξ)
end

# Test objective function at σ=0
f_gmm_rclogit_σ(W=W_test, Z=Z_test, X_all=test_data[1], σ=zeros(6)', s_all=test_data[2], ζ=ζ_test, I_num=I_test)

12.478842752838842

In [129]:
# Test with expanded instrument set
X_no_inst = reduce(vcat, test_data[1])[:,2:5]
Z_full = hcat(X_no_inst, Z_test)
W_full = inv(Z_full' * Z_full)

8×8 Array{Float64,2}:
  0.0270145    -0.00252669   -0.0123232    …   0.00142429   -0.000182233
 -0.00252669    0.000462362   0.00104184      -0.00110521   -0.000979029
 -0.0123232     0.00104184    0.00800275      -0.000905501   0.000122849
 -0.00548432    0.000391539   0.00220076      -0.000198159  -0.000220188
  0.00129346   -0.00119579   -0.000752589      0.000954218   0.000431562
  0.00046296   -0.000979566  -0.000563544  …  -0.000260153  -0.000354972
  0.00142429   -0.00110521   -0.000905501      0.0203489    -0.000635167
 -0.000182233  -0.000979029   0.000122849     -0.000635167   0.0202974

In [130]:
f_gmm_rclogit_σ(W=W_full, Z=Z_full, X_all=test_data[1], σ=zeros(6)', s_all=test_data[2], ζ=ζ_test, I_num=I_test)

0.5660004999500263

## Question 12

##### Write gradient function (as a function of $\sigma$ only).

In [79]:
# Individual share predictor function (given X and αβ)
function sHat_ind_αβ(; αβ, X, σ, ζ=0.0, I_n=1)
    # Construct numerators
    δ = X * αβ
    const_term = repeat(δ', outer=I_n)
    rc_term = ζ * (X .* repeat(σ, outer=length(δ)))'
    exp_term = const_term .+ rc_term
    max_exp = maximum(exp_term, dims=2)
    num = exp.(exp_term - repeat(max_exp, inner=(1,length(δ))))

    # Construct denominators and shares
    denom = repeat(exp.(-max_exp) .+ sum(num, dims=2), inner=(1,length(δ)))
    ŝ_i = num ./ denom
    return ŝ_i
end


# Function to compute Jacobian of ξ(θ) for each market
function ξ_Jac_θ_mkt(; X, I_n, ζ, ŝ_i)
    sum_terms = Array{Float64}(undef, size(X)[1], size(X)[2], I_n)
    ξ_Ji = Array{Float64}(undef, size(ŝ_i)[2], size(ŝ_i)[2], I_n)
    for i in 1:I_n
        s = ŝ_i[i,:]
        ξ_Ji[:,:,i] = -1.0 * ((s * s') .- diagm(vec(s)))
        sum_terms[:,:,i] = ξ_Ji[:,:,i] * X * diagm(ζ[i,:])
    end
    mean_all = 1/I_n * reshape(sum(sum_terms, dims=3), size(X)[1], size(X)[2])
    ξ_J_all = 1/I_n * reshape(sum(ξ_Ji, dims=3), size(ŝ_i)[2], size(ŝ_i)[2])
    return -1.0 * inv(ξ_J_all) * mean_all
end


# Function for computing Jacobian of ξ(θ) for all markets
function ξ_Jac(; X_all, δ_all, I_n, ζ, σ)
      ξ_J_θ = []
      for mkt in 1:length(X_all)
            X_mkt = X_all[mkt]; δ_mkt = δ_all[mkt]

            ŝ_i = sHat_ind(δ=δ_mkt, X=X_mkt, σ=σ, ζ=ζ, I_n=I_n)
            ξ_J_θ_mkt = ξ_Jac_θ_mkt(X=X_mkt, I_n=I_n, ζ=ζ, ŝ_i=ŝ_i)
            push!(ξ_J_θ, ξ_J_θ_mkt)
      end
      ξ_J = reduce(vcat, ξ_J_θ)
      return ξ_J
end

ξ_Jac (generic function with 1 method)

##### Test function for Jacobian (first compute $\delta$ values)

In [90]:
ξδθ_output =  ξδθ_σ_rclogit(X_all=test_data[1], σ=zeros(6)', s_all=test_data[2], ζ=ζ_test, I_num=25, 
        Z=Z_test, W=W_test)

ξ_Jac_test = ξ_Jac(X_all=test_data[1], δ_all=ξδθ_output.δ_all, I_n=I_test, ζ=ζ_test, σ=zeros(6)');

In [91]:
ξ_Jac_test[1:8,:]

8×6 Array{Float64,2}:
 -0.15068   0.00965054  -0.387707  -0.272062     -1.38202e-20  -1.67896e-15
 -0.151175  0.00965054  -0.646178  -0.272062      2.0302e-19   -1.67896e-15
 -0.179996  0.00965054  -0.387707  -0.272062      0.0247634    -1.42985e-15
 -0.193396  0.00965054  -0.646178  -0.272062      2.15891e-19   0.05055
 -0.389419  0.00965054  -0.969268  -0.272062     -2.35198e-21   0.05055
 -0.401599  0.00965054  -1.29236   -0.272062      1.26032e-20   0.05055
 -0.273641  0.00965054  -1.29236    6.75655e-15   9.77764e-20   0.05055
 -0.635625  0.00965054  -1.55083    6.86423e-15  -1.06189e-19   0.05055

In [114]:
function f_gmm_rclogit_σ_g(; X_all, δ_all, Z, W, σ, I_n, ζ, ξ)
      ξ_J = ξ_Jac(X_all=X_all, δ_all=δ_all, I_n=I_n, ζ=ζ, σ=σ)
      df_θ = 2 * (Z' * ξ_J)' * W * (Z' * ξ)
      return df_θ
end

f_gmm_rclogit_σ_J (generic function with 1 method)

In [120]:
my_test_gmm_J = f_gmm_rclogit_σ_J(X_all=test_data[1], δ_all=ξδθ_output.δ_all, I_n=I_test, ζ=ζ_test, 
        σ=zeros(6)', Z=Z_test, W=W_test, ξ=ξδθ_output.ξ)
my_test_gmm_J'

1×6 Adjoint{Float64,Array{Float64,1}}:
 -60.1842  49.1075  352.707  2.33519  3.25786  15.231

##### Test Jacobian of market shares using ForwardDiff

In [138]:
function gmm_σ_test(σ_init) 
    func = f_gmm_rclogit_σ(W=W_test, Z=Z_test, X_all=test_data[1], σ=σ_init,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

gmm_σ_test (generic function with 1 method)

In [139]:
fd_test_gmm_J = ForwardDiff.gradient(gmm_σ_test, zeros(6)')
fd_test_gmm_J

1×6 Adjoint{Float64,Array{Float64,1}}:
 1.33603  7.07581  -26.7085  -0.092702  -0.0558999  -1.87277

In [132]:
fd_test_gmm_J

1×6 Adjoint{Float64,Array{Float64,1}}:
 1.33603  -26.7085  0.0  0.0  0.0  0.0

# FIX THIS STILL

## Question 13

##### Write GMM function that only takes in the desired values of $\sigma$ (for random coefficients on price and engine size)

In [135]:
function gmm_σ(σ_init) 
    σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
    func = f_gmm_rclogit_σ(W=W_test, Z=Z_test, X_all=test_data[1], σ=σ_mod,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

gmm_σ (generic function with 1 method)

In [136]:
# Test GMM function
gmm_test(zeros(6)')

12.478842752838842

## TEMPORARILY - construct auto-diff function as placeholder for coded-up one

In [134]:
# Construct auto-diff function as placeholder
g_gmm_σ_auto = x -> ForwardDiff.gradient(gmm_σ, x)
g_gmm_σ_auto([1, 1])

2-element Array{Float64,1}:
     1.4401349487344577
 -9924.5984750484

##### Write function for calling one stage of GMM for a given weighting matrix.

In [141]:
function gmm_stage(; W, Z, X_all, s_all, ζ, I_num, σ_init)
    # Objective function
    function f_obj(σ_init) 
        σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
        func = f_gmm_rclogit_σ(W=W, Z=Z, X_all=X_all, σ=σ_mod, s_all=s_all, ζ=ζ, I_num=I_num)
        return func
    end
    
    # Gradient function
#     function g_obj(σ_init)
#         σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
#         grad = f_gmm_rclogit_σ_g(W=W, Z=Z, X_all=X_all, σ=σ_mod, s_all=s_all, ζ=ζ, I_num=I_num) 
#         G[1:2] = grad[1,3]
#     end
    
    # AD gradient function
    # g_obj = x -> ForwardDiff.gradient(f_obj, x)
    
    # Return result
    σ_result = Optim.minimizer(optimize(f_obj, g_obj, σ_init, BFGS(), Optim.Options(show_trace=true)))
    return σ_result
end  

gmm_stage (generic function with 1 method)

##### Write functions for implementing two-stage GMM.

In [ ]:
# Function for optimal weighting matrix
function gmm_opt_weight(; Z, ξ)
    n = size(Z)[1]
    Zξ = Z' * ξ
    return inv((1/n) * Zξ * Zξ')
end


# Function for gradient of ξ with respect to all parameters - STILL NEED TO FINISH THIS!!!!!!!!!!!!!!!!!!!!!!!!!!
function ξ_θfull_J(; Z, ξ)
    n = size(Z)[1]
    Zξ = Z' * ξ
    return inv((1/n) * Zξ * Zξ')
end


# Function for two-stage GMM
function two_stage_gmm(; W_stage1, Z, X_all, s_all, ζ, I_num, σ_init)
    # First stage estimation with given starting weighting matrix
    σ_stage1 = gmm_stage(W=W_stage1, Z=Z, X_all=X_all, s_all=s_all, ζ=ζ, I_num=I_num, σ_init=σ_init)
    
    # Compute optimal weighting matrix for second stage
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ_stage1, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W_stage1)
    ξ_stage1 = ξδθ_output.ξ
    W_stage2 = gmm_opt_weight(Z=Z, ξ=ξ_stage1)
    
    # Second stage estimation with given starting weighting matrix
    σ_stage2 = gmm_stage(W=W_stage2, Z=Z, X_all=X_all, s_all=s_all, ζ=ζ, I_num=I_num, σ_init=σ_init)
    
    # Compute covariance matrix for point estimates
    G = ξ_θfull_J(Z, ξ) # STILL NEED TO FINISH THIS HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ_stage1, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W_stage1)
    ξ_stage1 = ξδθ_output.ξ
    W_stage2 = gmm_opt_weight(Z=Z, ξ=ξ_stage1)
    Σ = inv(G' * inv(W_stage2) * G)